In [ ]:
# code for converting bddk100 into custom

import os
from PIL import Image

# Define directory path
dir_path = "/content/drive/MyDrive/lane_d/Target_dataset/train"
i=1;
# Loop through files in directory
for file_name in os.listdir(dir_path):
    # Check if file is a regular file (not a directory)
    if os.path.isfile(os.path.join(dir_path, file_name)):
        # Do something with file (e.g. print its name)
        
         print(i)
         i+=1
         # Load image
         img = Image.open(dir_path+'/'+file_name)
         rgb = img.convert('RGB')
         # Define list of colors to keep
         keep_colors = [(0,0,142),(0,0,7),(128,64,128)]  # Example colors
         # Loop through each pixel and change color if needed
         for y in range(rgb.height):
             for x in range(rgb.width):
                 pixel_color = rgb.getpixel((x, y))
                 if pixel_color not in keep_colors:
                     rgb.putpixel((x, y), (0,0,0))  # Change color to black


        
         new_image_path=os.path.join('/content/drive/MyDrive/lane_d/target_dataset_1/train',file_name)
         rgb.save(new_image_path) 

In [ ]:
# dataset unzip from drive
!unzip /content/drive/MyDrive/lane_d.zip

In [4]:
# removing file without target image
import os
import tensorflow as tf
image_files = sorted([os.path.join("/content/lane_d/target_dataset_1/train", file) for file in os.listdir('/content/lane_d/target_dataset_1/train') if file.endswith('_train_color.png')])
for img in image_files :
    file_name = tf.strings.regex_replace(img, '_train_color.png', '.jpg').numpy().decode()
    if  tf.io.gfile.exists(file_name) == False :
            os.remove(img)
            print(img)

/content/lane_d/target_dataset_1/train/fee92217-63b3f87f_train_color.png
/content/lane_d/target_dataset_1/train/ff1e4d6d-f4d85cfd_train_color.png
/content/lane_d/target_dataset_1/train/ff3d3536-04986e25_train_color.png
/content/lane_d/target_dataset_1/train/ff3da814-c3463a43_train_color.png


In [5]:

image_files = sorted([os.path.join("/content/lane_d/target_dataset_1/val", file) for file in os.listdir('/content/lane_d/target_dataset_1/val') if file.endswith('_train_color.png')])
for img in image_files :
    file_name = tf.strings.regex_replace(img, '_train_color.png', '.jpg').numpy().decode()
    if  tf.io.gfile.exists(file_name) == False :
            os.remove(img)
            print(img)

/content/lane_d/target_dataset_1/val/97b65dd9-48538198_train_color.png


In [6]:
# input pipiline

from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if  f.endswith("_train_color.png")]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        target_name = self.image_files[idx]
        img_name = target_name.replace("_train_color.png", ".jpg")
        img_path = os.path.join(self.root_dir, img_name)
        target_path = os.path.join(self.root_dir, target_name)
        img = Image.open(img_path).convert("RGB")
        target = Image.open(target_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
            target = self.transform(target)
        return img, target


In [7]:
# creating dataloder

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    #transforms.Lambda(lambda x: x / 255)
])
dataset = CustomDataset("/content/lane_d/target_dataset_1/train", transform=transform)
val_dataset = CustomDataset("/content/lane_d/target_dataset_1/val", transform=transform)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=32, shuffle=True)

In [37]:
#architecture implementation
import torch
import torch.nn as nn

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of the network
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of the network
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        # Downward path
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)

        # Upward path
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[-(idx//2+1)]
            x = torch.cat([x, skip_connection], dim=1)
            x = self.ups[idx+1](x)

        x = self.final_conv(x)
        return x


In [38]:
learning_rate = 0.001
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
model = UNet().to(device)

In [40]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [41]:
import torch.nn.functional as F

In [42]:
# loss function
def combined_loss(y_pred, y_true):
    ce_loss = F.cross_entropy(y_pred, y_true)
    mse_loss = F.mse_loss(y_pred, y_true)
    return ce_loss + 0.1* mse_loss

In [43]:
# load trained model
model.load_state_dict(torch.load('/content/drive/MyDrive/light_unet_lane_detection_last_epoch.pt',map_location=torch.device('cpu')),strict=False)

<All keys matched successfully>

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    
    # Train the model
    for i, batch in enumerate(dataloader):
        # Get the inputs and move them to the GPU
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = combined_loss(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics and log to TensorBoard
        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Epoch {epoch + 1}, batch {i + 1}: loss {running_loss / 100:.4f}")

            running_loss = 0.0
            v_losses = 0
            for val_batch in val_dataloader :
              v_inputs,v_labels = val_batch
              v_inputs = v_inputs.to(device)
              v_labels = v_labels.to(device)
              v_outputs = model(v_inputs)
              v_loss = combined_loss(v_outputs,v_labels)
              v_losses += v_loss.item()
              break
            print(f'val_loss : {v_losses}')
    # Save the model checkpoint
    torch.save(model.state_dict(), f"/content/drive/MyDrive/kaggle/light_unet_lane_detection_last_epoch.pt")


Epoch 1, batch 100: loss 0.3058
val_loss : 0.30661216378211975


In [2]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
#save ground truth,input,output

import torch
import torchvision
import torchvision.transforms as transforms

# Set the model to evaluation mode
model.eval()

save_path='/content/drive/MyDrive/result_1';


# Iterate over the validation dataset and make predictions
for i, (inputs,_) in enumerate(test_dataloader):
    # Pass the inputs through the model
    outputs = model(inputs)

    # Save the input and output images
    for j in range(len(inputs)):
        # input_image.save(os.path.join(save_path, f'input_{i}.jpg'))
        torchvision.utils.save_image(inputs[j], os.path.join(save_path, f'input_{i * len(inputs) + j}.png'))
        torchvision.utils.save_image(outputs[j],  os.path.join(save_path,f'output_{i * len(inputs) + j}.png'))
        torchvision.utils.save_image(labels[j],  os.path.join(save_path,f'label_{i * len(inputs) + j}.png'))
        

In [ ]:
# unzip test data from drive
!unzip /content/drive/MyDrive/test.zip

In [66]:
# Testing code
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

# Define the transforms to be applied to the input image
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load the input image

input_dir = '/content/test'
output_dir = '/content/drive/MyDrive/result_2'
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the input image
        image_path = os.path.join(input_dir, filename)
        input_image = Image.open(input_path).convert('RGB')

        # Apply the transforms to the input image
        input_tensor = transform(input_image).unsqueeze(0)

        # Apply the model to the input tensor
        with torch.no_grad():
            output = model(input_tensor)

        # Convert the output tensor to an image
        output_image = transforms.ToPILImage()(output.squeeze(0))

# Define the output path and save the output image to that directory
        output_path = os.path.join(output_dir, filename)
        output_image.save(output_path)
        